In [ ]:
import sys
sys.path.append('/tf/data')

import torch
from general_func import load_dataset
from Classification.class_functions import split_ds, concat_data
from Classification.Custom_GridSearch import grid_search

In [ ]:
params = {
    'images_root_path': '/tf/data/augmented_64_3ch/',
    'run': 'Search_1',
    'model_save_path': '/tf/data/Classification/ConvNeXt/Grid_Search3/',
    'model_save_freq_epochs': 1000,
    'seed': 42,

    'num_epochs': 150,
    'learning_rate': 4e-6,
    'weight_decay': 0.05,
    'warmup_epochs': 10,
    'early_stop': 15,

    'batch_size': 32,

    'loader_workers': 2,

    'drop_rate': 0.5,
    'apply_class_weights': True,

    #RandAUG
    'num_ops': 8,
    'magnitude': 20,

    #GridMask
    'offset': True, #False: square = 0, True: Square = noise
    'ratio': 0.5, #how much image to keep
    'mode': 1, #0 = keep squares, 1 = cut squares
    'prob': 0.5 #Probability to apply transformation 
    }

print('Device:', torch.device("cuda" if torch.cuda.is_available() else "cpu"))

In [ ]:
#Predict all validation data = negative = 75,38% accuracy (196/260)
#Load data and create dataloaders
ds_pos = load_dataset(positive=True)
train_scans_pos, val_scans_pos, test_scans_pos = split_ds(ds_pos, train_split = 0.7, val_split = 0.3, seed = params['seed'])
ds_neg = load_dataset(positive=False)
train_scans_neg, val_scans_neg, test_scans_neg = split_ds(ds_neg, train_split = 0.7, val_split = 0.3, seed = params['seed'])
search_n = 0
for batch in [8, 16, 32, 64]:
    params['batch_size'] = batch
    train_loader = concat_data(train_scans_pos, train_scans_neg, batch_size=params['batch_size'], workers=params['loader_workers'])
    val_loader = concat_data(val_scans_pos, val_scans_neg, batch_size=params['batch_size'], workers=params['loader_workers'])
    test_loader = concat_data(test_scans_pos, test_scans_neg, batch_size=params['batch_size'], workers=params['loader_workers'])

    for lr in [4e-2, 4e-3, 4e-4, 4e-5]:
        for weight_decay in [0.01, 0.005, 0.001]:
            for drop_rate in [0.5, 0.7]:
                search_n += 1
                params['learning_rate'] = lr
                params['weight_decay'] = weight_decay
                params['drop_rate'] = drop_rate
                grid_search(params, search_n, train_loader, val_loader, test_loader, ds_pos, ds_neg, log_path = params['model_save_path']+params['run'])